# Tut4 - Automation. Using Access Token Authentication and Updating Domo via API

In this tutorial we will generate an access_token and update an Account Object in Domo

## 🎓 What is Automation or Integration?

> "I want to generate and store my access_token in Domo Accounts"

Writing Integrations = decomposing a multiple step process into smaller pieces.

1. Use the UI to write down how you might approach this task by hand.
2. Each mouse-click often corresponds to an API request. Stub out some functions to capture the activities you're taking
3. Monitor network traffic to confirm assumptions.

### Possible Solution

1. Monitor network traffic to discover how to `get_all_access_tokens`, `generate_access_token`, `revoke_access_token`. Our goal is to create a function `regenerate_access_token` that can periodically revoke and reissue an updated access_token. Take note of the request body for any DELETE, POST, or PUT requests.

2. To add a layer of logic over all these base functions, Write a function to `upsert_access_token` that accepts `user_id`, `token_name` and `duration` then either creates a new access token or revokes and regenerates an existing token

3. Capture the result in a `domo_access_token` account type or `abstract_credentials_store` account type.
   - write a function to `get_all_accounts` (or search_accounts)
   - write a function to `create_domo_account`
   - write a function to `update_domo_account`
   - write a function to `update_domo_account_config` (note the process of updating Domo Account metadata is different from updating the configuration)
   - wrap it all in a function `upsert_account`

🎓 Theoretically, access_token based authentication is 'better' than username & password flow because it's easy to revoke an access_token without disrupting the end user experience. Also, because a user can have multiple access_tokens it allows administrators to have more fine grained control if revoking API based access becomes necessary


### Utils


In [1]:
import datetime as dt


def convert_domo_utc_to_datetime(timestamp):
    return dt.datetime.fromtimestamp(timestamp / 1000, dt.timezone.utc)

In [2]:
# Domo returns token expiration date in unix time

convert_domo_utc_to_datetime(1904914053000)

datetime.datetime(2030, 5, 13, 14, 47, 33, tzinfo=datetime.timezone.utc)

In [3]:
import time


def generate_domo_expiration_unixtimestamp(
    duration_in_days: int = 15, debug_prn: bool = False
):
    """# the expiration date of the access token is calculated based off of x days into the future which must then be converted into a unix timestamp"""

    today = dt.datetime.today()
    expiration_date = today + dt.timedelta(days=duration_in_days)

    if debug_prn:
        print(f"expiration_date is {duration_in_days} from today {expiration_date}")

    return int(time.mktime(expiration_date.timetuple()) * 1000)

In [4]:
# the expiration date of the access token is calculated based off of x days into the future which must then be converted into a unix timestamp

generate_domo_expiration_unixtimestamp(duration_in_days=15)

1712540367000

# Auth


In [5]:
from dotenv import load_dotenv

load_dotenv(".env")

True

In [6]:
from solutions.auth import get_session_token
import os

DOMO_INSTANCE = "domo-community"
DOMO_USERNAME = os.environ["DOMO_USERNAME"]
DOMO_PASSWORD = os.environ["DOMO_PASSWORD"]

assert DOMO_USERNAME and DOMO_PASSWORD


def get_instance_session_token(domo_username, domo_password, domo_instance):
    return get_session_token(
        domo_instance=domo_instance,
        domo_password=domo_password,
        domo_username=domo_username,
    )


test_session_token = get_instance_session_token(
    domo_username=DOMO_USERNAME,
    domo_password=DOMO_PASSWORD,
    domo_instance=DOMO_INSTANCE,
)

assert test_session_token

# Access Tokens


In [7]:
from  solutions.access_tokens import get_all_access_tokens

# import requests
# from solutions.client import ResponseClass
# from solutions.utils import convert_domo_utc_to_datetime


# def get_all_access_tokens(
#     domo_instance,
#     session_token=None,
#     headers=None,
#     return_raw: bool = False,
#     debug_api: bool = False,
# ):

#     url = f"https://{domo_instance}.domo.com/api/data/v1/accesstokens"

#     headers = headers or {}

#     if session_token:
#         headers.update({"x-domo-authentication": session_token})

#     if debug_api:
#         print({"url": url, "headers": headers})

#     res = requests.request(
#         url=url,
#         method="GET",
#         headers=headers,
#     )

#     res = ResponseClass.from_request_response(res)

#     if return_raw:
#         return res

#     [
#         token.update({"expires": convert_domo_utc_to_datetime(token["expires"])})
#         for token in res.response
#     ]

#     return res

In [8]:
import pandas as pd

res = get_all_access_tokens(
    session_token=test_session_token,
    domo_instance=DOMO_INSTANCE,
    return_raw=True,
    debug_api=False,
)
test_access_tokens = res.response
pd.DataFrame(test_access_tokens)[0:5]

,id,name,ownerId,ownerName,ownerEmail,expires
0,156180,Governance,587894148,None,None,1904914053000
1,159191,Java CLI,1345408759,None,None,1911327081000
2,163868,Governance,1345408759,None,None,1920056760000
3,168840,Dataset Copy,1345408759,None,None,1929380114000
4,182879,S3 Export,68216396,None,None,1665197515000


In [9]:
def search_tokens_by_user_id_and_token_name(
    session_token, domo_instance, user_id, token_name
):
    res = get_all_access_tokens(
        session_token=session_token,
        domo_instance=domo_instance,
        return_raw=True,
        debug_api=False,
    )

    all_tokens = res.response

    return next(
        (
            token
            for token in all_tokens
            if token["name"] == token_name and token["ownerId"] == user_id
        ),
        None,
    )


test_token = search_tokens_by_user_id_and_token_name(
    session_token=test_session_token,
    domo_instance=DOMO_INSTANCE,
    user_id=85654293,
    token_name="dp24",
)

test_token

{'id': 187009,
 'name': 'dp24',
 'ownerId': 85654293,
 'ownerName': None,
 'ownerEmail': None,
 'expires': 1712537469000}

# Users


In [10]:
from solutions.users import get_user_by_id

# def get_user_by_id(
#     domo_instance,
#     user_id: int,
#     session_token=None,
#     headers=None,
#     return_raw: bool = False,
#     debug_api: bool = False,
# ):
#     url = f"https://{domo_instance}.domo.com/api/content/v2/users/{user_id}"

#     headers = headers or {}

#     if session_token:
#         headers.update({"x-domo-authentication": session_token})

#     if debug_api:
#         print({"url": url, "headers": headers})

#     res = requests.request(
#         url=url,
#         method="GET",
#         headers=headers,
#     )

#     res = ResponseClass.from_request_response(res)

#     if return_raw:
#         return res

#     return res

In [11]:
get_user_by_id(
    domo_instance=DOMO_INSTANCE,
    session_token=test_session_token,
    user_id=85654293,
    debug_api=False,
)

ResponseClass(status=200, is_success=True, response={'id': 85654293, 'invitorUserId': 1893952720, 'displayName': 'test_and_delete', 'userName': 'test29@test.com', 'emailAddress': 'dp24@test.com', 'accepted': True, 'userType': 'USER', 'modified': 1711055143017, 'created': 1695810562, 'active': True, 'anonymous': True, 'systemUser': False, 'pending': True})

In [12]:
from solutions.access_tokens import generate_access_token

from pprint import pprint


# def generate_access_token(
#     domo_instance,
#     token_name: str,
#     user_id,
#     duration_in_days: 15,
#     session_token=None,
#     headers=None,
#     return_raw: bool = False,
#     debug_api: bool = False,
# ):
#     url = f"https://{domo_instance}.domo.com/api/data/v1/accesstokens"

#     expiration_timestamp = generate_domo_expiration_unixtimestamp(
#         duration_in_days=duration_in_days
#     )

#     body = {"name": token_name, "ownerId": user_id, "expires": expiration_timestamp}

#     headers = headers or {}

#     if session_token:
#         headers.update({"x-domo-authentication": session_token})

#     if debug_api:
#         pprint({"url": url, "headers": headers, "body": body})

#     res = requests.request(method="post", url=url, json=body, headers=headers)

#     res = ResponseClass.from_request_response(res)
#     if return_raw:
#         return res

#     return res

In [13]:
generate_access_token(
    domo_instance=DOMO_INSTANCE,
    session_token=test_session_token,
    token_name="dp24",
    duration_in_days=15,
    user_id=85654293,
    debug_api=False,
)

ResponseClass(status=200, is_success=True, response={'id': 187016, 'name': 'dp24', 'ownerId': 85654293, 'ownerName': 'test_and_delete', 'ownerEmail': 'dp24@test.com', 'expires': 1712540369000, 'token': '9f2134452e356b880e802f29b015e1cd58a489d686a96b18'})

In [17]:
from solutions.access_tokens import revoke_access_token
# def revoke_access_token(
#     domo_instance,
#     access_token_id,
#     session_token=None,
#     headers=None,
#     return_raw: bool = False,
#     debug_api: bool = False,
# ):

#     url = f"https://{domo_instance}.domo.com/api/data/v1/accesstokens/{access_token_id}"

#     headers = headers or {}

#     if session_token:
#         headers.update({"x-domo-authentication": session_token})

#     if debug_api:
#         pprint({"url": url, "headers": headers})

#     res = requests.request(method="DELETE", url=url, headers=headers)

#     res = ResponseClass.from_request_response(res)
#     if return_raw:
#         return res

#     res.response = f"access token {access_token_id} deleted"
#     return res


generate_access_token(
    domo_instance=DOMO_INSTANCE,
    session_token=test_session_token,
    token_name="dp24",
    duration_in_days=15,
    user_id=85654293,
    debug_api=False,
)

test_token = search_tokens_by_user_id_and_token_name(
    session_token=test_session_token,
    domo_instance=DOMO_INSTANCE,
    user_id=85654293,
    token_name="dp24",
)

if not test_token:
    raise Exception("no token")

# res =revoke_access_token(domo_instance = DOMO_INSTANCE,
#                     session_token=test_session_token,
#                     access_token_id=test_token['id'],
#                     return_raw=False,
#                     debug_api=False)

test_token

{'id': 187009,
 'name': 'dp24',
 'ownerId': 85654293,
 'ownerName': None,
 'ownerEmail': None,
 'expires': 1712537469000}

# Domo Accounts


In [18]:
from solutions.accounts import get_accounts_v2

# def get_accounts_v2(domo_instance, session_token, debug_api: bool = False):

#     url = f"https://{domo_instance}.domo.com/api/data/v1/accounts"

#     headers = {"x-domo-authentication": session_token}

#     if debug_api:
#         pprint({"url": url, "headers": headers})

#     res = requests.request(method="GET", url=url, headers=headers)

#     res = ResponseClass.from_request_response(res)

#     if not res.is_success:
#         raise DomoAPIRequest_Error(get_all_domo_accounts.__name__, res)

#     return res

In [19]:
res = get_accounts_v2(
    domo_instance=DOMO_INSTANCE, session_token=test_session_token
)
pd.DataFrame(res.response[0:5])

,id,userId,name,displayName,type,valid,dataProviderType,credentialsType,createdAt,createdBy,modifiedAt,modifiedBy,configurations,accountId,accountTemplateId,accountTemplateAuthorizationId
0,1,1893952720,DataSet Copy Account,dsa - northshore,data,True,dataset-copy,fields,1589100087000,1893952720,1698071224000,1893952720,{},1,None,None
1,27,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,domo-csv,fields,1619083568000,1893952720,1619083568000,1893952720,{},27,None,None
2,45,1893952720,onyxReporting@gmail.com,test-goolesheets,data,True,google-spreadsheets,oauth,1664924354000,1893952720,1682350828000,1893952720,{},45,None,None
3,70,1893952720,jaemyong.wilson@sony.com,jaemyong.wilson@sony.com,data,True,google-spreadsheets,oauth,1682511651000,1893952720,1682511651000,1893952720,{},70,None,None
4,71,1893952720,domo_creds,domolibrary test account - updated 2024-03-23,data,True,abstract-credential-store,fields,1684447092000,1893952720,1711223529000,1893952720,{},71,None,None


In [21]:
from solutions.accounts import generate_account__abstract_credential_store_body
# import datetime as dt
# import json


# def generate_account__abstract_credential_store_body(
#     account_name, credentials, is_timestamp: bool = True
# ):

#     # the API expects credentials to be type string, so conditionally convert dict to string
#     if isinstance(credentials, dict):
#         credentials = json.dumps(credentials)

#     if is_timestamp:
#         account_name = f"{account_name} - updated {dt.date.today()}"
#     return {
#         "name": "Abstract Credential Store Account",
#         "displayName": account_name,
#         "dataProviderType": "abstract-credential-store",
#         "configurations": {"credentials": credentials},
#     }


generate_account__abstract_credential_store_body(
    "test", "my creds go here", is_timestamp=True
)

{'name': 'Abstract Credential Store Account',
 'displayName': 'test - updated 2024-03-23',
 'dataProviderType': 'abstract-credential-store',
 'configurations': {'credentials': 'my creds go here'}}

In [22]:
from solutions.accounts import generate_account__access_token_body
# def generate_account__access_token_body(
#     account_name, access_token=None, username=None, password=None
# ):
#     return {
#         "name": "Domo Access Token Account",
#         "displayName": account_name,
#         "dataProviderType": "domo-access-token",
#         "configurations": {
#             "domoAccessToken": {access_token},
#             "username": {username},
#             "password": {password},
#         },
#     }


generate_account__access_token_body(
    account_name="jw_dp24", access_token="123", username="jae@test.com", password=123
)

{'name': 'Domo Access Token Account',
 'displayName': 'jw_dp24',
 'dataProviderType': 'domo-access-token',
 'configurations': {'domoAccessToken': {'123'},
  'username': {'jae@test.com'},
  'password': {123}}}

In [23]:
from solutions.accounts import create_account 
#  import requests
# from solutions.client import DomoAPIRequest_Error


# def create_account(body, domo_instance, session_token, debug_api: bool = False):
#     url = f"https://{domo_instance}.domo.com/api/data/v1/accounts"

#     headers = {"x-domo-authentication": session_token}

#     if debug_api:
#         pprint({"url": url, "headers": headers})

#     res = requests.request(method="POST", url=url, json=body, headers=headers)

#     res = ResponseClass.from_request_response(res)

#     if not res.is_success:
#         raise DomoAPIRequest_Error(create_account.__name__, res)

#     return res


create_account_body = generate_account__abstract_credential_store_body(
    account_name="my_domo_community_access_token", credentials=test_token
)

create_account(
    body=create_account_body,
    domo_instance=DOMO_INSTANCE,
    session_token=test_session_token,
)

ResponseClass(status=200, is_success=True, response={'id': 105, 'userId': 1893952720, 'name': 'my_domo_community_access_token - updated 2024-03-23 (5)', 'displayName': 'my_domo_community_access_token - updated 2024-03-23 (5)', 'type': 'data', 'valid': False, 'dataProviderType': 'abstract-credential-store', 'credentialsType': 'fields', 'createdAt': None, 'createdBy': 1893952720, 'modifiedAt': None, 'modifiedBy': 1893952720, 'configurations': {}, 'accountId': 105, 'accountTemplateId': None, 'accountTemplateAuthorizationId': None})

In [25]:
from solutions.accounts import get_account_by_id
# def get_account_by_id(
#     domo_instance,
#     account_id,
#     session_token=None,
#     headers=None,
#     return_raw: bool = False,
#     debug_api: bool = False,
# ):
#     url = f"https://{domo_instance}.domo.com/api/data/v1/accounts/{account_id}?unmask=true"

#     headers = headers or {}
#     headers.update({"x-domo-authentication": session_token})

#     if debug_api:
#         pprint({"url": url, "headers": headers})

#     res = requests.request(method="GET", url=url, headers=headers)

#     res = ResponseClass.from_request_response(res)

#     if return_raw:
#         return res

#     return res

In [26]:
get_account_by_id(
    domo_instance=DOMO_INSTANCE,
    account_id=102,
    session_token=test_session_token,
    debug_api=True,
)

{'headers': {'x-domo-authentication': 'eyJjdXN0b21lcklkIjoibW1tbS0wMDEyLTAyMDAiLCJleHBpcmF0aW9uIjoxNzExMjczMTY4MjE0LCJobWFjU2lnbmF0dXJlIjoiY2EyOGFiOTgzZjNmMzU4MTFmY2Y4MTdmN2NlMTg5YmU0YzljYzU0ODdmOWI1MTMwZWI3M2QyNGM4YjIxMTY3MCIsInNpZCI6IjcwYmI4YTczLWI1MjEtNGE5OS1hMTU5LTMwNmZkN2FmMWMxMCIsInRpbWVzdGFtcCI6MTcxMTI0NDM2ODIxNCwidG9lcyI6IlVOS05PV05TSUQiLCJ1c2VySWQiOiIxODkzOTUyNzIwIn0%3D'},
 'url': 'https://domo-community.domo.com/api/data/v1/accounts/102?unmask=true'}


ResponseClass(status=200, is_success=True, response={'id': 102, 'userId': 1893952720, 'name': 'my_domo_community_access_token - updated 2024-03-23 (2)', 'displayName': 'dp24 - update 2024-03-23 19:29:47.931225', 'type': 'data', 'valid': True, 'dataProviderType': 'abstract-credential-store', 'credentialsType': 'fields', 'createdAt': 1711242208000, 'createdBy': 1893952720, 'modifiedAt': 1711243788000, 'modifiedBy': 1893952720, 'configurations': {}, 'accountId': 102, 'accountTemplateId': None, 'accountTemplateAuthorizationId': None})

In [29]:
from solutions.accounts import update_account_config

# def update_account_config(
#     domo_instance,
#     dataprovider_type,
#     account_id,
#     body: dict,  # only receives configuration portion of the account definition
#     headers: dict = None,
#     session_token: str = None,
#     debug_api: bool = False,
#     return_raw: bool = False,
# ):
#     url = f"https://{domo_instance}.domo.com/api/data/v1/providers/{dataprovider_type}/account/{account_id}"

#     headers = headers or {}
#     headers.update({"x-domo-authentication": session_token})

#     if debug_api:
#         pprint({"url": url, "headers": headers, "body": body})

#     res = requests.request(method="PUT", json=body, url=url, headers=headers)

#     res = ResponseClass.from_request_response(res)

#     if return_raw:
#         return res

#     return res

In [30]:
create_account_body = generate_account__abstract_credential_store_body(
    account_name=f"my_domo_community_access_token - {dt.datetime.now()}",
    credentials=test_token,
)

# NOTE
config = create_account_body["configurations"]

update_account_config(
    domo_instance=DOMO_INSTANCE,
    account_id=102,
    dataprovider_type="abstract-credential-store",
    body=config,
    session_token=test_session_token,
    debug_api=True,
)

{'body': {'credentials': '{"id": 187009, "name": "dp24", "ownerId": 85654293, '
                         '"ownerName": null, "ownerEmail": null, "expires": '
                         '1712537469000}'},
 'headers': {'x-domo-authentication': 'eyJjdXN0b21lcklkIjoibW1tbS0wMDEyLTAyMDAiLCJleHBpcmF0aW9uIjoxNzExMjczMTY4MjE0LCJobWFjU2lnbmF0dXJlIjoiY2EyOGFiOTgzZjNmMzU4MTFmY2Y4MTdmN2NlMTg5YmU0YzljYzU0ODdmOWI1MTMwZWI3M2QyNGM4YjIxMTY3MCIsInNpZCI6IjcwYmI4YTczLWI1MjEtNGE5OS1hMTU5LTMwNmZkN2FmMWMxMCIsInRpbWVzdGFtcCI6MTcxMTI0NDM2ODIxNCwidG9lcyI6IlVOS05PV05TSUQiLCJ1c2VySWQiOiIxODkzOTUyNzIwIn0%3D'},
 'url': 'https://domo-community.domo.com/api/data/v1/providers/abstract-credential-store/account/102'}


ResponseClass(status=200, is_success=True, response={'id': 102, 'userId': 1893952720, 'name': 'my_domo_community_access_token - updated 2024-03-23 (2)', 'displayName': 'dp24 - update 2024-03-23 19:29:47.931225', 'type': 'data', 'valid': True, 'dataProviderType': 'abstract-credential-store', 'credentialsType': 'fields', 'createdAt': 1711242208000, 'createdBy': 1893952720, 'modifiedAt': 1711243788000, 'modifiedBy': 1893952720, 'configurations': {}, 'accountId': 102, 'accountTemplateId': None, 'accountTemplateAuthorizationId': None})

In [31]:
from solutions.accounts import update_account_name

# def update_account_name(
#     domo_instance: str,
#     account_id: int,
#     account_name: str,
#     session_token=None,
#     headers: dict = None,
#     debug_api: bool = False,
#     return_raw: bool = False,
# ):
#     url = f"https://{domo_instance}.domo.com/api/data/v1/accounts/{account_id}/name"

#     headers = headers or {}
#     headers.update({"x-domo-authentication": session_token})

#     if debug_api:
#         pprint({"url": url, "headers": headers, "body": account_name})

#     res = requests.request(method="PUT", data=account_name, url=url, headers=headers)

#     res = ResponseClass.from_request_response(res)

#     if return_raw:
#         return res

#     return res

In [ ]:
update_account_name(
    account_id=102,
    account_name=f"dp24 - update {dt.datetime.now()}",
    domo_instance=DOMO_INSTANCE,
    session_token=test_session_token,
    debug_api=True,
)

{'body': 'dp24 - update 2024-03-23 19:29:47.931225',
 'headers': {'x-domo-authentication': 'eyJjdXN0b21lcklkIjoibW1tbS0wMDEyLTAyMDAiLCJleHBpcmF0aW9uIjoxNzExMjcyNTgyNjA4LCJobWFjU2lnbmF0dXJlIjoiNDczNDI5YmU0YzBhOWU5MTQ3ODg2Yjk0ODgwMzcxMjM4ZTAyYjlkODk0OWQ0MTVjM2Q2ZmIwNzY3ODY0MTAzZCIsInNpZCI6IjhjNjI0YTZhLWE3MzYtNGNkZi1hZTEwLWJhY2Y5MzNiZTYxOCIsInRpbWVzdGFtcCI6MTcxMTI0Mzc4MjYwOCwidG9lcyI6IlVOS05PV05TSUQiLCJ1c2VySWQiOiIxODkzOTUyNzIwIn0%3D'},
 'url': 'https://domo-community.domo.com/api/data/v1/accounts/102/name'}


ResponseClass(status=200, is_success=True, response={'id': 102, 'userId': 1893952720, 'name': 'my_domo_community_access_token - updated 2024-03-23 (2)', 'displayName': 'dp24 - update 2024-03-23 19:29:47.931225', 'type': 'data', 'valid': True, 'dataProviderType': 'abstract-credential-store', 'credentialsType': 'fields', 'createdAt': 1711242208000, 'createdBy': 1893952720, 'modifiedAt': 1711243788000, 'modifiedBy': 1893952720, 'configurations': {}, 'accountId': 102, 'accountTemplateId': None, 'accountTemplateAuthorizationId': None})